# INSY 5336 Python Programming Spring 2021 Final Term Project 

Algorithm:
1. import urllib.request, urllib.parse, urllib.error for opening urls to retrieve data, BeautifulSoup for web scrapping by parsing html/xml, ssl for secure communication, csv for reading and writing to csv files.
2. Create a default context object for secure communication.
3. Define get_stock_data_and_save() function which takes category name and category's dictionary as input. In the function: 
    i) Open a stocks.csv in append mode as we would have already written the headers in the file in the main() function.
    ii) Iterate through the category's dictionary to get the quote for each ticker symbol from its detail page using BeautifulSoup and save the details along with stock name in stocks.csv
    iii) close the csv file.
4. Define main() function.In the function: 
    i) Open the https://money.cnn.com/data/hotstocks/ url using urllib.request.urlopen()
    ii) Create the BeautifulSoup object
    iii) Get the list of Most Actives, Gainers and Losers from https://money.cnn.com/data/hotstocks/ website using the soup object.
    iv) Write the headers to the csv file in the first row
    v) Call get_stock_data_and_save() function for Most Actives, Gainers and Losers.
    vi) Print the list of Most Actives, Gainers and Losers.
    vii) Get data from stocks.csv and save it in a dictionary.
    viii) Ask the user to enter the ticker symbol to print its details. If user does not enter a valid ticker, print error and ask him to enter it again.
    ix)  If user enters correct ticker, print the details.
5. Call main().

Instructions to run the code: (The program takes a while to run. Please be patient)
1. Execute the cell below.
2. First the program will scrape the https://money.cnn.com/data/hotstocks/ website to get a list of Most Actives, Gainers and Losers.
3. Then for each stock in Most Actives, Gainers and Losers, stock's detail page will be scraped to get open, close, volume, market cap and this data will be saved in stocks.csv
4. Then user will be asked to input a ticker symbol to print its details.If user does not enter a valid ticker, error will be printed and he will be asked to enter it again. If user enters correct ticker, details of that stock will be printed.

Program:

In [2]:
# for opening urls to retrieve data
import urllib.request, urllib.parse, urllib.error
# for web scrapping by parsing html/xml
from bs4 import BeautifulSoup
# to use OpenSSL for secure communication
import ssl
# for reading and writing to csv files
import csv

# create default context for secure communication
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
  
# this function take a dictionary as input and gets the details of each ticker symbol in the dictionary using its url and saves the details in stocks.csv
def get_stock_data_and_save(category,dict_name):
    # open csv file named stocks.csv in append mode as the headers are already written to the csv
    csv_file = open('stocks.csv', 'a', newline = '')
    csv_writer = csv.writer(csv_file)
    # get the quote for each ticker symbol in the input dictionary and save the details along with stock name in stocks.csv
    # one row per ticker symbol in the csv
    for sym,value in dict_name.items():
        # url to get the quote for the ticker symbol
        detail_page_url = 'https://money.cnn.com'+value[1]
        # open the url for scrapping quote for the ticker symbol
        detail_page_html = urllib.request.urlopen(detail_page_url, context=ctx).read() 
        # create the BeautifulSoup object
        detail_page_soup = BeautifulSoup(detail_page_html, 'lxml')
        # find h3 whose text is 'Today’s Trading'
        detail_page_header = detail_page_soup.find('h3', text = 'Today’s Trading')
        # use find_next_sibling() method to get the table for the quote as its tag is parallel to h3 tag
        detail_page_table = detail_page_header.find_next_sibling('div').find('table')
        # get open price
        todays_open = detail_page_table.find('td', text = 'Today’s open').find_next_sibling('td').text
        # get previous close
        prev_close = detail_page_table.find('td', text = 'Previous close').find_next_sibling('td').text
        # get volume
        volume = detail_page_table.find('td', text = 'Volume').find_next_sibling('td').text
        # get market cap
        market_cap = detail_page_table.find('td', text = 'Market cap').find_next_sibling('td').text
        # write the details to csv in a row for this ticker symbol
        csv_writer.writerow([category,sym, value[0], todays_open, prev_close, volume, market_cap])
    #close the csv file
    csv_file.close()
      
# this function gets the list of Most Actives, Gainers and Losers from https://money.cnn.com/data/hotstocks/ website and prints it
# it then calls get_stock_data_and_save() function to save the details for each stock
# it asks the user to input a ticker symbol and if user enters correct ticker, print the details
def main():
    # open the url for scrapping Most Actives, Gainers and Losers
    url = 'https://money.cnn.com/data/hotstocks/'
    html = urllib.request.urlopen(url, context=ctx).read() 
    # create the BeautifulSoup object
    soup = BeautifulSoup(html, 'lxml')

    # Code to scrape Most Actives
    # find h3 whose text is 'Most Actives'
    header = soup.find('h3', text = 'Most Actives')
    # use find_next_sibling() method to get the table for Most Actives as its tag is parallel to the above h3 tag
    table = header.find_next_sibling('table')
    # define a list to store the Most Actives stock names with the ticker symbol
    MostActives_list = []
    # define a dictionary with key as Most Actives stock's ticker symbol and value as a tuple of stock's title and detailed page url
    MostActives_dict = {}
    # iterate through all the trs for this table to get the Most Actives stocks
    table_rows = table.find_all('tr')
    for tr in table_rows:
        if tr.find('td') != None:
            td = tr.find('td')
            # append stock names with the ticker symbol
            MostActives_list.append(td.text)
            # get the ticker symbol
            ticker_symbol = td.a.text
            # get the url of the detail page
            ticker_link = td.a.get('href')
            # get the stock title
            ticker_title = td.span.get('title')
            # add to the dictionary with key as Most Actives stock's ticker symbol and value as a tuple of stock's title and detail page url
            MostActives_dict[ticker_symbol] = (ticker_title,ticker_link)
            
    # Code to scrape Gainers
    # find h3 whose text is 'Gainers'
    header = soup.find('h3', text = 'Gainers')
    # use find_next_sibling() method to get the table for Gainers as its tag is parallel to the above h3 tag
    table = header.find_next_sibling('table')
    # define a list to store the Gainers stock names with the ticker symbol
    Gainers_list = []
    # define a dictionary with key as Gainers stock's ticker symbol and value as a tuple of stock's title and detailed page url
    Gainers_dict = {}
    # iterate through all the trs for this table to get the Gainers stocks
    table_rows = table.find_all('tr')
    for tr in table_rows:
        if tr.find('td') != None:
            td = tr.find('td')
            # append stock names with the ticker symbol
            Gainers_list.append(td.text)
            # get the ticker symbol
            ticker_symbol = td.a.text
            # get the url of the detail page
            ticker_link = td.a.get('href')
            # get the stock title
            ticker_title = td.span.get('title')
            # add to the dictionary with key as Gainers stock's ticker symbol and value as a tuple of stock's title and detail page url
            Gainers_dict[ticker_symbol] = (ticker_title,ticker_link)
            
    # Code to scrape Losers
    # find h3 whose text is 'Losers'
    header = soup.find('h3', text = 'Losers')
    # use find_next_sibling() method to get the table for Losers as its tag is parallel to the above h3 tag
    table = header.find_next_sibling('table')
    # define a list to store the Losers stock names with the ticker symbol
    Losers_list = []
    # define a dictionary with key as Losers stock's ticker symbol and value as a tuple of stock's title and detailed page url
    Losers_dict = {}
    # iterate through all the trs for this table to get the Losers stocks
    table_rows = table.find_all('tr')
    for tr in table_rows:
        if tr.find('td') != None:
            td = tr.find('td')
            # append stock names with the ticker symbol
            Losers_list.append(td.text)
            # get the ticker symbol
            ticker_symbol = td.a.text
            # get the url of the detail page
            ticker_link = td.a.get('href')
            # get the stock title
            ticker_title = td.span.get('title')
            # add to the dictionary with key as Losers stock's ticker symbol and value as a tuple of stock's title and detail page url
            Losers_dict[ticker_symbol] = (ticker_title,ticker_link)
         
    # open csv file named stocks.csv in write mode
    csv_file = open('stocks.csv', 'w', newline = '')
    csv_writer = csv.writer(csv_file)
    # write the headers to the csv file in the first row
    csv_writer.writerow(['Category','Ticker Symbol', 'Stock Name', 'Open', 'Prev Close', 'Volume', 'Market Cap'])
    #close the csv file
    csv_file.close()    
    # call get_stock_data_and_save() function to get details for each Most Actives stock, Gainers stock and Losers stock and save it in csv
    get_stock_data_and_save("Most Actives",MostActives_dict)
    get_stock_data_and_save("Gainers",Gainers_dict)
    get_stock_data_and_save("Losers",Losers_dict)
    # ask the user for the ticker symbol to view the details
    print('Which stock are you interested in:\n')
    # print the Most Actives stocks with their ticker symbol
    print('Most Actives:')
    for stock in MostActives_list:
        print(stock)
    print('\n')
    # print the Gainers stocks with their ticker symbol
    print('Gainers:')
    for stock in Gainers_list:
        print(stock)
    print('\n')
    # print the Losers stocks with their ticker symbol
    print('Losers:')
    for stock in Losers_list:
        print(stock)
    print('\n')
    
    # open stocks.csv to get all the data from the csv file into a dictionary with key as ticker symbol and value as its detail
    csv_file = open('stocks.csv')
    csv_reader = csv.reader(csv_file, delimiter=',')
    csv_data = {}
    for line in csv_reader:
        csv_data[line[1]] = {}
        csv_data[line[1]]['title'] = line[1]+" "+line[2]
        csv_data[line[1]]['open'] = line[3]
        csv_data[line[1]]['prev_close'] = line[4]
        csv_data[line[1]]['volume'] = line[5]
        csv_data[line[1]]['market_cap'] = line[6]
    # close the csv
    csv_file.close()
    # if user does not enter a valid ticker, print error and ask him to enter it again
    while True:
        user_input = input('User inputs: ')
        # remove whitespaces from user_input
        user_input = user_input.upper().strip()
        if user_input not in csv_data:
            print("Please enter a valid stock ticker!")
        else:
            # if user enters correct ticker, print the details
            print("The data for",csv_data[user_input]['title'],"is the following:")
            print(csv_data[user_input]['title'])
            print('OPEN: ',csv_data[user_input]['open'])
            print('PREV CLOSE: ',csv_data[user_input]['prev_close'])
            print('VOLUME: ',csv_data[user_input]['volume'])
            print('MARKET CAP: ',csv_data[user_input]['market_cap'])
            break

# call the main function to begin execution
main()

Which stock are you interested in:

Most Actives:
GE General Electric Co
F Ford Motor Co
BAC Bank of America Corp
PFE Pfizer Inc
T AT&T Inc
CCL Carnival Corp
TWTR Twitter Inc
FCX Freeport-McMoRan Inc
WFC Wells Fargo & Co
XOM Exxon Mobil Corp


Gainers:
IT Gartner Inc
SEE Sealed Air Corp
LEG Leggett & Platt Inc
BEN Franklin Resources Inc
AMCR Amcor PLC
MLM Martin Marietta Materials Inc
VMC Vulcan Materials Co
NUE Nucor Corp
EL Estee Lauder Companies Inc
CVS CVS Health Corp


Losers:
CTLT Catalent Inc
PAYC Paycom Software Inc
CCL Carnival Corp
RCL Royal Caribbean Cruises Ltd
DAL Delta Air Lines Inc
KR Kroger Co
GNRC Generac Holdings Inc
LYV Live Nation Entertainment Inc
GPN Global Payments Inc
TYL Tyler Technologies Inc


User inputs: twt
Please enter a valid stock ticker!
User inputs: twtr
The data for TWTR Twitter Inc is the following:
TWTR Twitter Inc
OPEN:  55.07
PREV CLOSE:  54.58
VOLUME:  30,404,484
MARKET CAP:  $44.1B
